In [12]:
import os
import sys
import numpy as np
import random

In [2]:
import torch
from torch.autograd import Variable

In [3]:
import tensorflow as tf
from tensorflow import keras

2024-01-31 14:59:13.675344: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-31 14:59:13.695395: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 14:59:13.695414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 14:59:13.695954: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 14:59:13.699408: I tensorflow/core/platform/cpu_feature_guar

In [4]:
from pytorch2keras import pytorch_to_keras
from onnx2keras import onnx_to_keras
import onnx
# import onnxoptimizer as optimizer

In [5]:
sys.path.append("../../")

In [6]:
from th.resources import models
from th.resources import calculator as calc

In [7]:
src_pt_model_path = "../retrained_models_after_pruned/retrained_model_20240124123209_acc_95.45455169677734_795th_epoch.pt"
random_weigths_pruned_keras_model_path = "../keras_models/pruned_model_copy_weights_from_retrained_model_20240124123209_acc_95.45455169677734_v2.keras"

In [8]:
state = torch.load(src_pt_model_path)
torch_model_weights = state['weight']
config = state['config']
test_torch_model = models.GetACDNetModel(input_len=30225, nclass=2, sr=20000, channel_config=config)
test_torch_model.load_state_dict(torch_model_weights)

<All keys matched successfully>

In [9]:
random_weigths_pruned_keras_model = keras.models.load_model(random_weigths_pruned_keras_model_path)

2024-01-31 14:59:17.180143: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 14:59:17.180742: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 14:59:17.180804: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [13]:
seed = 42;
random.seed(seed);
np.random.seed(seed);
torch.manual_seed(seed);
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed);
torch.backends.cudnn.deterministic = True;
torch.backends.cudnn.benchmark = False;

In [25]:
# input_np = np.random.uniform(0, 1, (1,1,1,30225))
# print(input_np)
input_torch_var = torch.randn(1,1,1,30225, requires_grad=True)#Variable(torch.FloatTensor(input_np))
input_np = input_torch_var.detach().numpy();

input_tf_var = tf.convert_to_tensor(np.transpose(input_np,(1,2,3,0)))
print(input_tf_var.shape)

(1, 1, 30225, 1)


In [28]:
tf_res = random_weigths_pruned_keras_model.predict(input_tf_var)
print(tf_res.shape)

1/1 [==============================] - 0s 11ms/step
(1, 2)


In [29]:
torch_res = test_torch_model(input_torch_var)
print(torch_res.shape)

torch.Size([1, 2])


In [17]:
random_weigths_pruned_keras_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1, 30225, 1)]     0         
                                                                 
 conv2d_12 (Conv2D)          (None, 1, 15109, 5)       45        
                                                                 
 batch_normalization_12 (Ba  (None, 1, 15109, 5)       20        
 tchNormalization)                                               
                                                                 
 re_lu_12 (ReLU)             (None, 1, 15109, 5)       0         
                                                                 
 conv2d_13 (Conv2D)          (None, 1, 7553, 32)       800       
                                                                 
 batch_normalization_13 (Ba  (None, 1, 7553, 32)       128       
 tchNormalization)                                         

In [229]:
calc.summary(torch_model, (1,1,30225))

+----------------------------------------------------------------------------+
+                           Pytorch Model Summary                            +
------------------------------------------------------------------------------
   Layer (type)       Input Shape      Output Shape    Param #      FLOPS #
       Conv2d-1     (1, 1, 30225)     (5, 1, 15109)         45      679,905
  BatchNorm2d-2     (5, 1, 15109)     (5, 1, 15109)         10            0
         ReLu-3     (5, 1, 15109)     (5, 1, 15109)          0       75,545
       Conv2d-4     (5, 1, 15109)     (32, 1, 7553)        800    6,042,400
  BatchNorm2d-5     (32, 1, 7553)     (32, 1, 7553)         64            0
         ReLu-6     (32, 1, 7553)     (32, 1, 7553)          0      241,696
    MaxPool2d-7     (32, 1, 7553)      (32, 1, 151)          0      241,600
      Permute-8      (32, 1, 151)      (1, 32, 151)          0            0
       Conv2d-9      (1, 32, 151)      (9, 32, 151)         81      391,392
 Ba

## pytorch conv2d NCHW || tensorflow conv2d NHWC

In [233]:
t2t_conv_pos_mov = (2,3,1,0)
# t2t_conv_pos_mov = (1,2,3,0)

In [234]:
# set the conv2d_12 weights
tmp_w = torch.permute(torch_model_weights['sfeb.0.weight'],t2t_conv_pos_mov)
random_weigths_pruned_keras_model.get_layer('conv2d_12').set_weights([tmp_w.cpu()])

In [235]:
#set the batch_normalization_12
tmp_w1 = torch_model_weights['sfeb.1.weight'].cpu()
tmp_w2 = torch_model_weights['sfeb.1.bias'].cpu()
tmp_w3 = torch_model_weights['sfeb.1.running_mean'].cpu()
tmp_w4 = torch_model_weights['sfeb.1.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_12').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [236]:
#set the conv2d_13 weights
print(torch_model_weights['sfeb.3.weight'].shape)
tmp_w = torch.permute(torch_model_weights['sfeb.3.weight'],(2,3,1,0)) 
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_13').set_weights([tmp_w.cpu()])

torch.Size([32, 5, 1, 5])
torch.Size([1, 5, 5, 32])


In [237]:
#set the batch_normalization_13
tmp_w1 = torch_model_weights['sfeb.4.weight'].cpu()
tmp_w2 = torch_model_weights['sfeb.4.bias'].cpu()
tmp_w3 = torch_model_weights['sfeb.4.running_mean'].cpu()
tmp_w4 = torch_model_weights['sfeb.4.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_13').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [239]:
#set the conv2d_14 weights
# Layer conv2d_14 weight shape (3, 3, 1, 9) is not compatible with provided weight shape torch.Size([9, 1, 3, 3]).
tmp_w = torch_model_weights['tfeb.0.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_14').set_weights([tmp_w.cpu()])

torch.Size([9, 1, 3, 3])
torch.Size([3, 3, 1, 9])


In [240]:
#set the batch_normalization_14
tmp_w1 = torch_model_weights['tfeb.1.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.1.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.1.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.1.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_14').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [241]:
#set the conv2d_15 weights
tmp_w = torch_model_weights['tfeb.4.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_15').set_weights([tmp_w.cpu()])

torch.Size([16, 9, 3, 3])
torch.Size([3, 3, 9, 16])


In [242]:
#set the batch_normalization_15
tmp_w1 = torch_model_weights['tfeb.5.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.5.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.5.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.5.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_15').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [243]:
#set the conv2d_16 weights
tmp_w = torch_model_weights['tfeb.7.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_16').set_weights([tmp_w.cpu()])

torch.Size([23, 16, 3, 3])
torch.Size([3, 3, 16, 23])


In [244]:
#set the batch_normalization_16
tmp_w1 = torch_model_weights['tfeb.8.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.8.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.8.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.8.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_16').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [245]:
#set the conv2d_17 weights
tmp_w = torch_model_weights['tfeb.11.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_17').set_weights([tmp_w.cpu()])

torch.Size([33, 23, 3, 3])
torch.Size([3, 3, 23, 33])


In [246]:
#set the batch_normalization_17
tmp_w1 = torch_model_weights['tfeb.12.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.12.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.12.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.12.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_17').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [247]:
#set the conv2d_18 weights
tmp_w = torch_model_weights['tfeb.14.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_18').set_weights([tmp_w.cpu()])

torch.Size([29, 33, 3, 3])
torch.Size([3, 3, 33, 29])


In [248]:
#set the batch_normalization_18
tmp_w1 = torch_model_weights['tfeb.15.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.15.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.15.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.15.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_18').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [249]:
#set the conv2d_19 weights
tmp_w = torch_model_weights['tfeb.18.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_19').set_weights([tmp_w.cpu()])

torch.Size([56, 29, 3, 3])
torch.Size([3, 3, 29, 56])


In [250]:
#set the batch_normalization_19
tmp_w1 = torch_model_weights['tfeb.19.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.19.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.19.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.19.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_19').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [251]:
#set the conv2d_20 weights
tmp_w = torch_model_weights['tfeb.21.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_20').set_weights([tmp_w.cpu()])

torch.Size([47, 56, 3, 3])
torch.Size([3, 3, 56, 47])


In [252]:
#set the batch_normalization_20
tmp_w1 = torch_model_weights['tfeb.22.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.22.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.22.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.22.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_20').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [253]:
#set the conv2d_21 weights
tmp_w = torch_model_weights['tfeb.25.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_21').set_weights([tmp_w.cpu()])

torch.Size([65, 47, 3, 3])
torch.Size([3, 3, 47, 65])


In [254]:
#set the batch_normalization_21
tmp_w1 = torch_model_weights['tfeb.26.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.26.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.26.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.26.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_21').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [255]:
#set the conv2d_22 weights
tmp_w = torch_model_weights['tfeb.28.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_22').set_weights([tmp_w.cpu()])

torch.Size([90, 65, 3, 3])
torch.Size([3, 3, 65, 90])


In [256]:
#set the batch_normalization_22
tmp_w1 = torch_model_weights['tfeb.29.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.29.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.29.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.29.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_22').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [257]:
#set the conv2d_23 weights
tmp_w = torch_model_weights['tfeb.33.weight']
print(tmp_w.shape)
tmp_w = torch.permute(tmp_w,t2t_conv_pos_mov)
print(tmp_w.shape)
random_weigths_pruned_keras_model.get_layer('conv2d_23').set_weights([tmp_w.cpu()])

torch.Size([2, 90, 1, 1])
torch.Size([1, 1, 90, 2])


In [258]:
#set the batch_normalization_23
tmp_w1 = torch_model_weights['tfeb.34.weight'].cpu()
tmp_w2 = torch_model_weights['tfeb.34.bias'].cpu()
tmp_w3 = torch_model_weights['tfeb.34.running_mean'].cpu()
tmp_w4 = torch_model_weights['tfeb.34.running_var'].cpu()
random_weigths_pruned_keras_model.get_layer('batch_normalization_23').set_weights([tmp_w1,tmp_w2,tmp_w3,tmp_w4])

In [259]:
#set the dense_1 weights
dense_w = torch_model_weights['tfeb.38.weight'].cpu()
dense_b = torch_model_weights['tfeb.38.bias'].cpu()
random_weigths_pruned_keras_model.get_layer('dense_1').set_weights([dense_w,dense_b])

In [260]:
random_weigths_pruned_keras_model.save("../keras_models/pruned_model_copy_weights_from_retrained_model_20240124123209_acc_95.45455169677734_v2.h5")
random_weigths_pruned_keras_model.save("../keras_models/pruned_model_copy_weights_from_retrained_model_20240124123209_acc_95.45455169677734_v2.keras")

/home/ai/miniconda3/envs/libtestenv/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# random_weigths_pruned_keras_model.get_layer('conv2d_13').weights

In [ ]:
# torch.permute(torch_model_weights['sfeb.3.weight'],(2,1,3,0))

In [ ]:
# torch_model_weights.keys()

In [ ]:
# calc.summary(torch_model, (1,1,30225));